In [1]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import psycopg2 as ps

In [2]:
# Importamos el csv con la información de las reservas, los hoteles y los clientes
data = pd.read_csv("../data/hoteles_unidos.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'], dtype={'id_cliente': str, 'id_hotel': str})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   nombre           15000 non-null  object        
 2   apellido         15000 non-null  object        
 3   mail             15000 non-null  object        
 4   competencia      15000 non-null  bool          
 5   fecha_reserva    15000 non-null  datetime64[ns]
 6   inicio_estancia  15000 non-null  datetime64[ns]
 7   final_estancia   15000 non-null  datetime64[ns]
 8   nombre_hotel     15000 non-null  object        
 9   ciudad           15000 non-null  object        
 10  id_cliente       15000 non-null  object        
 11  id_hotel         15000 non-null  object        
 12  estrellas        15000 non-null  float64       
 13  precio_medio     15000 non-null  float64       
dtypes: bool(1), datetime64[ns](3), float64

In [3]:
# Importamos el csv con los eventos de Madrid sacados de la API
eventos = pd.read_csv("../data/datos_eventos_limpio.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   nombre_evento      1149 non-null   object        
 1   url_evento         1149 non-null   object        
 2   codigo_postal      1149 non-null   int64         
 3   direccion          1026 non-null   object        
 4   horario            1149 non-null   object        
 5   organizacion       1041 non-null   object        
 6   fecha_inicio       1149 non-null   datetime64[ns]
 7   fecha_fin          1149 non-null   datetime64[ns]
 8   hora_fecha_inicio  1149 non-null   object        
 9   hora_fecha_fin     1149 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 89.9+ KB


In [4]:
# Nos conectamos a la bddd de postgres
conn = ps.connect(
    dbname="Hoteles_etl",
    user="postgres",
    password="quique",
    host = "localhost",
    port = "5432")

In [5]:
# Creamos un cursor con el objeto conexion
cur = conn.cursor()

In [6]:
# Ejecutamos para saber si estamos conectados
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit',)

In [7]:
# comprobamos que no hay valores duplicados
data.duplicated().sum()



np.int64(0)

In [31]:
data_insert_ciudad = ["Madrid"]

In [77]:
insert_query = """
INSERT INTO ciudad (nombre_ciudad) VALUES (%s)
"""

In [78]:
cur.execute(insert_query, data_insert_ciudad)
conn.commit()

In [79]:
# vamos a comprobar que se ha creado correctamente
query_ciudad = """
    SELECT * 
    FROM ciudad; 
"""
cur.execute(query_ciudad)
cur.fetchall()

[(1, 'Madrid')]

In [80]:
# Creamos la tabla eventos

cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad") # Seleccionamos la ciudad de Madrid
ciudades = dict(cur.fetchall()) # Creamos un diccionario con las ciudades
ciudades

{'Madrid': 1}

In [81]:
eventos.head(1)

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,fecha_inicio,fecha_fin,hora_fecha_inicio,hora_fecha_fin
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,Sin horario,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02,00:00:00,23:59:00


In [82]:
data_insert_eventos = [] # Creamos una lista vacía para insert

for i, row in eventos.iterrows():
    nombre_evento = row["nombre_evento"] 
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"] if row["codigo_postal"] != 0 else None
    direccion = row["direccion"] if pd.notna(row["direccion"]) else None 
    horario = row["horario"] if pd.notna(row["horario"]) else None
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"] if pd.notna(row["organizacion"]) else None
    id_ciudad = ciudades.get("Madrid")

    data_insert_eventos.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))

In [83]:
data_insert_eventos

[('25º aniversario de la revista La Fragua',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD',
  28005,
  'CALLE SAN JUSTO 5',
  'Sin horario',
  Timestamp('2025-02-24 00:00:00'),
  Timestamp('2025-03-02 00:00:00'),
  'Biblioteca Pública Municipal Iván de Vargas (Centro)',
  1),
 ('3CLONWS',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=fc8d040e7c784910VgnVCM1000001d4a900aRCRD',
  28012,
  'RONDA ATOCHA 35',
  '19:00',
  Timestamp('2025-03-21 00:00:00'),
  Timestamp('2025-03-22 00:00:00'),
  'Teatro Circo Price',
  1),
 ('60 Premio Reina Sofía de Pintura y Escultura',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
  28009,
  'PASEO COLOMBIA 1',
  'Sin horario',
  Timestamp('2025-02-27 00:00:00'),
  Timestamp('2025-03-23 00:00:

In [84]:
insert_query_eventos = """ 
                        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                        fecha_inicio, fecha_fin,organizacion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [85]:
cur.executemany(insert_query_eventos, data_insert_eventos)
conn.commit()

In [86]:
# Insertamos la tabla hoteles

data_insert_hoteles = []

for _, row in data.iterrows():
    id_hotel = row["id_hotel"]
    nombre_hotel = row["nombre_hotel"] 
    competencia = row["competencia"] 
    valoracion = row["estrellas"] 
    id_ciudad = ciudades.get("Madrid") 

    hoteles = (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)

    if hoteles not in data_insert_hoteles:
        data_insert_hoteles.append(hoteles)

In [87]:
data_insert_hoteles

[('1', 'Hotel Monte Verde', False, 3.1036363636363635, 1),
 ('2', 'Hotel Brisas del Mar', False, 3.088014981273408, 1),
 ('3', 'Hotel Camino del Sol', False, 3.0675675675675675, 1),
 ('4', 'Hotel Puerta del Cielo', False, 3.02851711026616, 1),
 ('5', 'Hotel Encanto Real', False, 3.031311154598826, 1),
 ('6', 'Palacio del Sol', False, 2.992156862745098, 1),
 ('7', 'Hotel Jardines del Rey', False, 2.9362549800796813, 1),
 ('8', 'Hotel Las Estrellas', False, 2.906614785992218, 1),
 ('9', 'Gran Hotel Madrid', False, 3.0675675675675675, 1),
 ('10', 'Hotel Torre Dorada', False, 2.919921875, 1),
 ('11', 'Hotel Palacio Imperial', False, 3.0064794816414686, 1),
 ('12', 'Hotel Luz de Madrid', False, 3.0522388059701493, 1),
 ('13', 'Hotel Los Almendros', False, 3.0112570356472794, 1),
 ('14', 'Hotel Sol y Luna', False, 3.0089766606822264, 1),
 ('15', 'Hotel Mirador Real', False, 2.9774859287054407, 1),
 ('16', 'Hotel Rincón Sereno', False, 2.998046875, 1),
 ('17', 'Hotel Vista Alegre', False, 2.9

In [88]:
# Creamos la query de inserción de los datos de hotel
insert_query_hoteles = """ 
                        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s)
"""

In [89]:
cur.executemany(insert_query_hoteles, data_insert_hoteles)
conn.commit()

In [96]:
# Creamos la tabla clientes

# revisar tipo de datos
data.columns

Index(['id_reserva', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'nombre_hotel',
       'ciudad', 'id_cliente', 'id_hotel', 'estrellas', 'precio_medio'],
      dtype='object')

In [91]:
data_insert_clientes = []

for _, row in data.iterrows():

    id_cliente = row["id_cliente"]
    nombre = row["nombre"],
    apellido = row["apellido"],
    mail = row["mail"]

    tabla_cliente = (id_cliente, nombre, apellido, mail)

    if tabla_cliente not in data_insert_clientes:
        data_insert_clientes.append(tabla_cliente)

In [92]:
# Creamos la query
insert_query_clientes = """ 
                        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
"""

In [93]:
cur.executemany(insert_query_clientes, data_insert_clientes)
conn.commit()

In [8]:
# Creamos la tabla reservas
data.columns

Index(['id_reserva', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'nombre_hotel',
       'ciudad', 'id_cliente', 'id_hotel', 'estrellas', 'precio_medio'],
      dtype='object')

In [9]:
# Sacamos el id de clientes de la tabla de clientes
cur.execute("SELECT nombre, id_cliente FROM clientes")
dictio_clientes = dict(cur.fetchall())
dictio_clientes

{'Maite': '10756',
 'Tecla': '15610',
 'Amílcar': '11807',
 'Joan': '15243',
 'Chelo': '9477',
 'Dolores': '11983',
 'Noé': '13720',
 'Omar': '15043',
 'Ágata': '13673',
 'Julie': '12728',
 'Javier': '3454',
 'Rita': '15447',
 'Ricardo': '14239',
 'Perlita': '15222',
 'Pedro': '15034',
 'Judith': '14008',
 'Francisco Javier': '14594',
 'Reynaldo': '14021',
 'Felicidad': '10432',
 'Pía': '14810',
 'Blas': '14297',
 'Claudia': '15715',
 'Fernanda': '11988',
 'Emilio': '15458',
 'Bárbara': '12615',
 'Emiliano': '14208',
 'Marisol': '13916',
 'Ezequiel': '13370',
 'Telmo': '14764',
 'Débora': '15180',
 'Darío': '14166',
 'Fausto': '15702',
 'Lorenza': '13112',
 'Anunciación': '14767',
 'María Belén': '12287',
 'Amaya': '13364',
 'Jose Angel': '14293',
 'Berta': '6046',
 'Salud': '15829',
 'Úrsula': '11973',
 'Sabas': '15363',
 'Asunción': '15210',
 'Jafet': '12275',
 'Adrián': '12933',
 'Benigno': '15193',
 'Jordán': '13548',
 'Sofía': '14963',
 'Ainara': '14144',
 'Segismundo': '13820',
 

In [10]:
data_insert_reservas = []

for _, row in data.iterrows():
    
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row["inicio_estancia"]
    final_estancia = row["final_estancia"]
    id_cliente = row["id_cliente"]
    id_hotel = row["id_hotel"]

    tabla_reservas = (id_reserva, fecha_reserva, inicio_estancia, final_estancia, id_cliente, id_hotel)

    if tabla_reservas not in data_insert_reservas:
        data_insert_reservas.append(tabla_reservas)

In [11]:
data_insert_reservas

[('40c4cb55-d1f5-407b-832f-4756b8ff77b4',
  Timestamp('2025-02-09 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-03 00:00:00'),
  '1002',
  '1'),
 ('f2ce8df5-7844-43e1-8c0f-97ce7a208f21',
  Timestamp('2025-02-08 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-03 00:00:00'),
  '1003',
  '2'),
 ('57d4515a-447d-4067-afcb-9bdcf4e4e915',
  Timestamp('2025-02-02 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-03 00:00:00'),
  '1004',
  '3'),
 ('ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7',
  Timestamp('2025-02-08 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-03 00:00:00'),
  '1005',
  '4'),
 ('99c3dc4f-663c-45f7-849a-ac9313f3746a',
  Timestamp('2025-02-09 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-03 00:00:00'),
  '1006',
  '5'),
 ('a5da0906-0fe5-4c30-b193-863df67a7b84',
  Timestamp('2025-02-10 00:00:00'),
  Timestamp('2025-03-01 00:00:00'),
  Timestamp('2025-03-03 00:00:00'),
  '1008',


In [12]:
# Creamos la query
insert_query_reservas = """ 
                        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query_reservas, data_insert_reservas)
conn.commit()

In [13]:
cur.close() 
conn.close()
